In [1]:
import numpy as np
import pandas as pd 
import sklearn
from sklearn.metrics  import f1_score,accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
# Classification
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier
#
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.metrics import *
from sklearn.model_selection import *
# from catboost import CatBoost,CatBoostRegressor
from xgboost import XGBRegressor,XGBRFRegressor
import warnings
warnings.filterwarnings('ignore')
np.random.seed(55)
sklearn.random.seed(55)

/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.5.2.1-4-gc0b48a7 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.13.1-unknown is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.36ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
data = pd.read_csv('./data/train.csv')

In [3]:
data = data.sample(frac=1)

In [4]:
test_data = pd.read_csv('./data/test.csv')

In [5]:
from tqdm import tqdm

In [6]:
data['HomePlanet'] = data['HomePlanet'].replace({"Earth":1,"Europa":2,"Mars":3})
home_plant_labels = {"Earth":1,"Europa":2,"Mars":3}
data['CryoSleep'] = data['CryoSleep'].replace({True:1,False:2})
cryo_sleep_labels = {"Earth":1,"Europa":2,"Mars":3}
cabins_1 = []
cabins_2 = []
cabins_3 = []
for i in tqdm(range(len(data['Cabin']))):
    cabin = str(data.iloc[i]['Cabin']).split("/")
    if cabin[0] != "nan":
        cabins_1.append(cabin[0])
        cabins_2.append(int(cabin[1]))
        cabins_3.append(cabin[2])
    else:
        cabins_1.append(None)
        cabins_2.append(None)
        cabins_3.append(None)
data['Cabins_1'] = cabins_1
data['Cabins_2'] = cabins_2
data['Cabins_3'] = cabins_3

100%|████████████████████████████| 8693/8693 [00:00<00:00, 15605.31it/s]


In [7]:
data['Transported'] =data['Transported'].astype('category').cat.codes
data['VIP'] =data['VIP'].astype('category').cat.codes
data['Cabins_1'] =data['Cabins_1'].astype('category').cat.codes
data['Cabins_3'] =data['Cabins_3'].astype('category').cat.codes
data.drop('Cabin',axis=1,inplace=True)
data.drop('Destination',axis=1,inplace=True)
data.drop('Name',axis=1,inplace=True)

In [8]:
data['PassengerId'] = data['PassengerId'].astype(int)

In [9]:
data['HomePlanet'] = data['HomePlanet'].fillna(data['HomePlanet'].mean())
data['CryoSleep'] = data['CryoSleep'].fillna(data['CryoSleep'].mean())
data['Age'] = data['Age'].fillna(data['Age'].mean())
data['RoomService'] = data['RoomService'].fillna(data['RoomService'].mean())
data['FoodCourt'] = data['FoodCourt'].fillna(data['FoodCourt'].mean())
data['ShoppingMall'] = data['ShoppingMall'].fillna(data['ShoppingMall'].mean())
data['Spa'] = data['Spa'].fillna(data['Spa'].mean())
data['VRDeck'] = data['VRDeck'].fillna(data['VRDeck'].mean())
data['Cabins_2'] = data['Cabins_2'].fillna(data['Cabins_2'].mean())

In [10]:
X = data.drop('Transported',axis=1)
y = data['Transported']

In [11]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125,shuffle=True)

In [12]:
num_columns = list(X.columns)
ct = make_column_transformer(
    (MinMaxScaler(), num_columns),
    (StandardScaler(), num_columns),
    remainder='passthrough'
)

X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier

In [14]:
params = {
    'C':[1.0,2.0,3.0,4.0,5],
    "kernel":["linear","poly","rbf","sigmoid"],
    "degree":[1,2,3,4,5],
    "gamma":["scale","auto"],
    "shrinking":[True,False],
    "cache_size":[12,25,50,100,250,325,500,625,750,1000],
    "max_iter":[1,2,3,3,4,5],
}

In [15]:
model = SVC()
gs = GridSearchCV(model,params,cv=3)
gs.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'C': [1.0, 2.0, 3.0, 4.0, 5],
                         'cache_size': [12, 25, 50, 100, 250, 325, 500, 625,
                                        750, 1000],
                         'degree': [1, 2, 3, 4, 5], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                         'max_iter': [1, 2, 3, 3, 4, 5],
                         'shrinking': [True, False]})

In [16]:
# {'C': 1.0,
#  'cache_size': 12,
#  'degree': 1,
#  'gamma': 'auto',
#  'kernel': 'sigmoid',
#  'max_iter': 5,
#  'shrinking': True}

In [20]:
model = SVC(
    C=1.0,
    cache_size=12,
    degree=1,
    gamma='auto',
    kernel='sigmoid',
    max_iter=5,
    shrinking=True
)
model.fit(X_train,y_train)

SVC(cache_size=12, degree=1, gamma='auto', kernel='sigmoid', max_iter=5)

In [21]:
gs.best_params_

{'C': 1.0,
 'cache_size': 12,
 'degree': 1,
 'gamma': 'auto',
 'kernel': 'sigmoid',
 'max_iter': 5,
 'shrinking': True}

In [22]:
model.score(X_test,y_test)

0.7019319227230911

In [23]:
preds = model.predict(X_test)

In [24]:
accuracy_score(preds,y_test),precision_score(preds,y_test),f1_score(preds,y_test)

(0.7019319227230911, 0.8788426763110307, 0.75)

In [25]:
np.mean(cross_val_score(model,X_test,y_test))

0.525273749630068

In [26]:
test_data = pd.read_csv('./data/test.csv')
ids = test_data["PassengerId"]
test_data['HomePlanet'] = test_data['HomePlanet'].replace({"Earth":1,"Europa":2,"Mars":3})
home_plant_labels = {"Earth":1,"Europa":2,"Mars":3}
test_data['CryoSleep'] = test_data['CryoSleep'].replace({True:1,False:2})
cryo_sleep_labels = {"Earth":1,"Europa":2,"Mars":3}
cabins_1 = []
cabins_2 = []
cabins_3 = []
for i in tqdm(range(len(test_data['Cabin']))):
    cabin = str(test_data.iloc[i]['Cabin']).split("/")
    if cabin[0] != "nan":
        cabins_1.append(cabin[0])
        cabins_2.append(int(cabin[1]))
        cabins_3.append(cabin[2])
    else:
        cabins_1.append(None)
        cabins_2.append(None)
        cabins_3.append(None)
test_data['Cabins_1'] = cabins_1
test_data['Cabins_2'] = cabins_2
test_data['Cabins_3'] = cabins_3
test_data['VIP'] =test_data['VIP'].astype('category').cat.codes
test_data['Cabins_1'] =test_data['Cabins_1'].astype('category').cat.codes
test_data['Cabins_3'] =test_data['Cabins_3'].astype('category').cat.codes
test_data.drop('Cabin',axis=1,inplace=True)
test_data.drop('Destination',axis=1,inplace=True)
test_data.drop('Name',axis=1,inplace=True)
test_data['PassengerId'] = test_data['PassengerId'].astype(int)
test_data['HomePlanet'] = test_data['HomePlanet'].fillna(test_data['HomePlanet'].mean())
test_data['CryoSleep'] = test_data['CryoSleep'].fillna(test_data['CryoSleep'].mean())
test_data['Age'] = test_data['Age'].fillna(test_data['Age'].mean())
test_data['RoomService'] = test_data['RoomService'].fillna(test_data['RoomService'].mean())
test_data['FoodCourt'] = test_data['FoodCourt'].fillna(test_data['FoodCourt'].mean())
test_data['ShoppingMall'] = test_data['ShoppingMall'].fillna(test_data['ShoppingMall'].mean())
test_data['Spa'] = test_data['Spa'].fillna(test_data['Spa'].mean())
test_data['VRDeck'] = test_data['VRDeck'].fillna(test_data['VRDeck'].mean())
test_data['Cabins_2'] = test_data['Cabins_2'].fillna(test_data['Cabins_2'].mean())
test_data = ct.transform(test_data)
preds = model.predict(np.array(test_data))
predictions = {
    "PassengerId":[],
    "Transported":[]
}
labels = {1:True,0:False}
for id,pred in zip(ids,preds):
    predictions['PassengerId'].append(id)
    predictions['Transported'].append(labels[pred])
predictions = pd.DataFrame(predictions)
predictions.to_csv(f'./save/final.csv',index=False)

100%|████████████████████████████| 4277/4277 [00:00<00:00, 14501.31it/s]


In [ ]:

# SVC() = 0.80593